In [1]:
import tweepy
import re
import json

import sqlite3 as lite

import pandas as pd

import datetime, time, os, sys
import geocoder
import requests
from pprint import pprint
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [2]:
# Load SQLite database
con = lite.connect("data/swcc.db")

# Import users into pd db
users = pd.read_sql("SELECT user_id, screen_name, user_object, fetch_followers, error, user_modified FROM users WHERE error IS NULL",
                    con,
                    index_col='user_id')

# Import followers into pd db
followers = pd.read_sql("SELECT user_id, follower_id, modified FROM followers",
                        con,
                        index_col='follower_id')

## Geocode the locations of users

In [3]:
users['location'] = users.user_object.map(lambda x: json.loads(x)['location'])

In [5]:
with requests.Session() as session:
    users['country_osm'] = users.location.progress_apply(lambda x: geocoder.osm(x, session=session).country if x != "" else None)


100%|██████████| 455/455 [03:55<00:00,  2.02it/s]


In [6]:
users.to_csv("data/users_with_location.csv")

## Export edge files for Networks (all users and CA only)

In [7]:
all_users = followers[followers.index.isin(users.index)]

edges = all_users[['user_id']]
edges.columns = ["Source"]
edges.index.name = "Target"
edges.to_csv("data/all_users_edges.csv")

In [9]:
nodes = users[['screen_name', 'country_osm']]
nodes.columns = ["Label", "Country"]
nodes.index.name = "Id"
nodes.to_csv("data/all_users_nodes.csv")

## Juan's Twitter Code

In [36]:
df = df[~df.tweet.isnull()]
df['tweet'] = df.tweet.apply(lambda x: json.loads(x) if x is not None else None)

df['created_at'] = df.tweet.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(x['created_at'],'%a %b %d %H:%M:%S +0000 %Y')))
df['created_at'] = pd.to_datetime(df.created_at)
df['created_at_dayofweek'] = df.tweet.apply(lambda x: x['created_at'][0:3])
df['user'] = df.tweet.apply(lambda x: x['user'])
df['screen_name'] = df.tweet.apply(lambda x: x['user']['screen_name'])
#     df['user_id'] = df.tweet.apply(lambda x: int(x['user']['id_str']))
#     df['user_utc_offset'] = df.tweet.apply(lambda x: x['user']['utc_offset'])
#     df['user_name'] = df.tweet.apply(lambda x: x['user']['name'])    
#     df['user_followers_count'] = df.tweet.apply(lambda x: x['user']['followers_count'])
#     df['user_friends_count'] = df.tweet.apply(lambda x: x['user']['friends_count'])
#     df['user_description'] = df.tweet.apply(lambda x: re.sub( '\s+', ' ', x['user']['description']).strip())
#     df['user_statuses_count'] = df.tweet.apply(lambda x: x['user']['statuses_count'])
df['is_retweet'] = df.tweet.apply(lambda x: 'retweeted_status' in x)
df['is_retweet'] = df['is_retweet'].fillna(False)
df['retweet_of_status_id_str'] = df.tweet.apply(lambda x: x['retweeted_status']['id_str'] if 'retweeted_status' in x else None)
df['retweet_of_screen_name'] = df.tweet.apply(lambda x: x['retweeted_status']['user']['screen_name'] if 'retweeted_status' in x else None)
df['is_reply'] = df.tweet.apply(lambda x: x['in_reply_to_status_id'] != None)
df['in_reply_to_status_id_str'] = df.tweet.apply(lambda x: x['in_reply_to_status_id_str'])
df['in_reply_to_screen_name'] = df.tweet.apply(lambda x: x['in_reply_to_screen_name'])
df['text'] = df.tweet.apply(lambda x: re.sub( '\s+', ' ', x['text']).strip()) # remove commas for CSV simplicity
del df['tweet']
tweetdetails = df.sort_index()
del df

df = pd.read_sql("SELECT doi, tweet_id, old_screen_name FROM sample WHERE error LIKE '%screen_name%'", litecon, index_col='old_screen_name')
users_df = pd.read_sql("SELECT screen_name, user_object FROM users", litecon, index_col='screen_name')
users_df['user'] = users_df.user_object.map(json.loads)
del users_df['user_object']

df = df.join(users_df, how="inner")
df.index.name = 'screen_name'  
df = df.reset_index().set_index('tweet_id')

tweetdetails = tweetdetails.append(df).sort_index()
del df

for field in ['id', 'name', 'followers_count', 'friends_count','statuses_count', 'description']:
    tweetdetails['user_%s' % field] = tweetdetails.user.map(lambda x: x[field])
del tweetdetails['user']